<a href="https://colab.research.google.com/github/ashpakshaikh26732/Unet-FCN/blob/main/Unet_FCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**all packages**

In [ ]:
import tensorflow as tf
import os

In [ ]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
  print(f'no of tpus : {strategy.num_replicas_in_sync}')
except ValueError :
  print('tpu failed to initilized')

no of tpus : 8


**mixed precision training**

In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**copy datasets from drive**

In [ ]:
!cp /content/drive/MyDrive/Cityscapes/gtFine_trainvaltest.zip /content/
!cp /content/drive/MyDrive/Cityscapes/leftImg8bit_trainvaltest.zip /content

**extract trainig the data**

In [ ]:
os.system("unzip -q /content/gtFine_trainvaltest.zip -d /content/ ")
os.system("unzip -q /content/leftImg8bit_trainvaltest.zip -d /content/")

256

In [ ]:
print("Train Images:", len(os.listdir("/content/leftImg8bit/train/aachen")))  # Adjust path as needed
print("Train Labels:", len(os.listdir("/content/gtFine/train/aachen")))

Train Images: 174
Train Labels: 696


In [ ]:
print("test images : ", len(os.listdir('/content/leftImg8bit/test')))

test images :  6


**Global Valrible**

In [ ]:
img_height = 224
img_width = 224
batch_size = 16
# final_batch_size = batch_size * strategy.num_replicas_in_sync

**loading images and labels from directry**

In [ ]:
def load_img():
    train_dir = '/content/leftImg8bit/train'
    train_image_cities = os.listdir(train_dir)
    train_images = []

    for city in train_image_cities:
        train_city = os.path.join(train_dir, city)
        images = [os.path.join(train_city, img) for img in os.listdir(train_city) if img.endswith('.png')]
        train_images.extend(images)

    return train_images

def load_labels():
    train_dir = '/content/gtFine/train'
    train_labels_cities = os.listdir(train_dir)
    train_labels = []
    for city in train_labels_cities:
        train_city_label = os.path.join(train_dir, city)
        labels = [f for f in os.listdir(train_city_label) if f.endswith('_gtFine_labelIds.png')]
        for label in labels:
            label_path = os.path.join(train_city_label, label)
            train_labels.append(label_path)
    return train_labels

**reading images from path**

In [ ]:
def read_img(img_path):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels=3)
    img = tf.image.resize(img, (img_height, img_width))
    img = img / 255.0
    return img

def read_labels(label_path):
    label = tf.io.read_file(label_path)
    label = tf.io.decode_png(label, channels=1)
    label = tf.image.resize(label, (img_height, img_width), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return tf.cast(label, tf.uint8)


**augmentation**

In [ ]:
def augment(image, label):

    label = tf.cast(label, tf.float32)

    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)
        label = tf.image.flip_left_right(label)

    image = tf.image.random_brightness(image, max_delta=0.1)

    combined = tf.concat([image, label], axis=-1)
    combined = tf.image.random_crop(combined, size=[224, 224, 6])

    image, label = tf.split(combined, num_or_size_splits=2, axis=-1)


    label = tf.cast(label, tf.uint8)

    return image, label


**creating dataset**

In [ ]:
images = load_img()
labels = load_labels()
dataset = tf.data.Dataset.from_tensor_slices((images, labels))
dataset = dataset.map(lambda image_path , label_path :( read_img(image_path) , label_path)).map(lambda image , label_path : (image,read_labels(label_path=label_path)))
dataset.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(buffer_size=1000).cache().batch(batch_size)
dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# for image , label in dataset.take(1):
#   print(f'image shape : {image.shape} , label shape {label.shape}')
#   print(f'image datatype : {image.dtype}, label dtype {label.dtype}')

**FCN model**

In [ ]:
class Block(tf.keras.models.Model):
    def __init__(self, n_conv, filters, kernel_size, activation, pool_size, pool_stride, block_name):
        super().__init__()
        self.conv_layers = [
            tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, padding='same',
                                   activation=activation, name=f"{block_name}_conv{i+1}")
            for i in range(n_conv)
        ]
        self.pool = tf.keras.layers.MaxPool2D(pool_size=pool_size, strides=pool_stride, name=f"{block_name}_pool")

    def call(self, inputs):
        x = inputs
        for conv in self.conv_layers:
            x = conv(x)
        x = self.pool(x)
        return x

In [ ]:
class Encoder(tf.keras.models.Model):
  def __init__(self):
    super().__init__()
    self.block1 = Block(n_conv=2 , filters=64 , kernel_size = (3,3) , activation ='relu' , pool_size=(2,2) , pool_stride = (2,2) , block_name = 'block1')
    self.block2 = Block(n_conv=3 , filters = 128 , kernel_size = (3,3) , activation='relu' , pool_size = (2,2) , pool_stride = (2,2) , block_name = 'block2')
    self.block3 = Block(n_conv=3 , filters = 256 , kernel_size = (3,3) , activation='relu' , pool_size = (2,2) , pool_stride = (2,2) , block_name = 'block3')
    self.block4 = Block(n_conv=3 , filters = 512 , kernel_size = (3,3) , activation='relu' , pool_size = (2,2) , pool_stride = (2,2) , block_name = 'block4')
    self.block5 = Block(n_conv=3 , filters = 512 , kernel_size = (3,3) , activation='relu' , pool_size = (2,2) , pool_stride = (2,2) , block_name = 'block5')
  def call(self,inputs):
    p1 = self.block1(inputs)
    p2 = self.block2(p1)
    p3 = self.block3(p2)
    p4 = self.block4(p3)
    p5 = self.block5(p4)
    return p1 , p2 , p3 , p4 , p5

In [ ]:
class Decoder(tf.keras.models.Model):
    def __init__(self, nclasses):
        super().__init__()


        self.upsample1 = tf.keras.layers.Conv2DTranspose(nclasses, kernel_size=(4, 4), strides=(2, 2), padding='same', use_bias=False)
        self.upsample2 = tf.keras.layers.Conv2DTranspose(nclasses, kernel_size=(4, 4), strides=(2, 2), padding='same', use_bias=False)
        self.upsample3 = tf.keras.layers.Conv2DTranspose(nclasses, kernel_size=(4, 4), strides=(2, 2), padding='same', use_bias=False)


        self.skip1 = tf.keras.layers.Conv2D(nclasses, kernel_size=(1, 1), activation='relu', padding='same')
        self.skip2 = tf.keras.layers.Conv2D(nclasses, kernel_size=(1, 1), activation='relu', padding='same')
        self.skip3 = tf.keras.layers.Conv2D(nclasses, kernel_size=(1, 1), activation='relu', padding='same')


        self.add1 = tf.keras.layers.Add()
        self.add2 = tf.keras.layers.Add()
        self.add3 = tf.keras.layers.Add()


        self.fcn32 = tf.keras.layers.Conv2DTranspose(nclasses, kernel_size=(32, 32), strides=(32, 32), padding='same', name='fcn32')
        self.fcn16 = tf.keras.layers.Conv2DTranspose(nclasses, kernel_size=(16, 16), strides=(16, 16), padding='same', name='fcn16')
        self.fcn8 = tf.keras.layers.Conv2DTranspose(nclasses, kernel_size=(8, 8), strides=(8, 8), padding='same', name='fcn8')
        self.fcn4 = tf.keras.layers.Conv2DTranspose(nclasses, kernel_size=(4, 4), strides=(4, 4), padding='same', name='fcn4')


        self.softmax = tf.keras.layers.Softmax(axis=-1)

    def call(self, inputs):
        p1, p2, p3, p4, p5 = inputs

        fcn32 = self.fcn32(p5)


        o = self.upsample1(p5)
        o = self.add1([o, self.skip1(p4)])
        fcn16 = self.fcn16(o)


        o = self.upsample2(o)
        o = self.add2([o, self.skip2(p3)])
        fcn8 = self.fcn8(o)


        o = self.upsample3(o)
        o = self.add3([o, self.skip3(p2)])
        fcn4 = self.fcn4(o)


        return self.softmax(fcn32), self.softmax(fcn16), self.softmax(fcn8), self.softmax(fcn4)

In [ ]:
class FCN(tf.keras.models.Model):
  def __init__(self):
    super().__init__()
    self.Encoder = Encoder()
    self.Decoder = Decoder(19)
  def call(self, inputs):
    x=self.Encoder(inputs)
    x = self.Decoder(x)
    return x

**UNET**

In [ ]:
class ConvBlock(tf.keras.models.Model):
  def __init__(self, n_filters,block_name , kernel_size=3 , strides = (1,1) , activation='relu'):
    super().__init__()
    self.conv = [tf.keras.layers.Conv2D(filters=n_filters , strides=strides , kernel_size=(kernel_size, kernel_size) , activation='relu', name=f"{block_name}_conv{i+1}", padding='same') for i in range (2)]
  def call(self, input):
    x = input
    for layer in self.conv:
      x = layer(x)
    return x

In [ ]:
class EncoderBlock(tf.keras.models.Model):
  def __init__(self, n_filters , pool_size , dropout,block_name):
    super().__init__()
    self.convBlock = ConvBlock(n_filters=n_filters , block_name=block_name, strides=(1,1))
    self.pool = tf.keras.layers.MaxPooling2D(pool_size=pool_size)
    self.dropout = tf.keras.layers.Dropout(rate = dropout)
  def call (self, input) :
    f = self.convBlock(input)
    p = self.pool(f)
    p = self.dropout(p)
    return f , p

In [ ]:
class Encoder (tf.keras.models.Model):
  def __init__(self):
    super().__init__()
    self.encoder1 = EncoderBlock(64 , pool_size=(2,2),dropout=0.3 , block_name = 'block1')
    self.encoder2 = EncoderBlock(128 , pool_size=(2,2),dropout=0.3 , block_name = 'block2')
    self.encoder3 = EncoderBlock(256 , pool_size=(2,2),dropout=0.3 , block_name = 'block3')
    self.encoder4 = EncoderBlock(512, pool_size=(2,2),dropout=0.3 , block_name = 'block4')
  def call(self, input):
    f1,p1=self.encoder1(input)
    f2,p2 = self.encoder2(p1)
    f3 , p3 = self.encoder3(p2)
    f4,p4 = self.encoder4(p3)
    return p4,(f1,f2,f3,f4)

In [ ]:
class BottleNeck(tf.keras.models.Model):
  def __init__(self):
    super().__init__()
    self.convBlock=ConvBlock(1024, block_name = 'bottleneck' )
  def call(self, input):
    x = self.convBlock(input)
    return x

In [ ]:
class DecoderBlock(tf.keras.models.Model):
  def __init__(self, n_filters ,dropout,block_name , kernel_size= (3,3),strides = (2,2)  ):
    super().__init__()
    self.convTranspose = tf.keras.layers.Conv2DTranspose(n_filters, kernel_size = kernel_size , strides=strides , padding = 'same')
    self.concat = tf.keras.layers.Concatenate()
    self.DropOut = tf.keras.layers.Dropout(dropout)
    self.convBlock = ConvBlock(n_filters=n_filters , block_name =block_name  )
  def call(self, inputs , convOutput):
    u=self.convTranspose(inputs)
    c = self.concat([u,convOutput])
    c= self.DropOut(c)
    c = self.convBlock(c)
    return c

In [ ]:
class Decoder (tf.keras.models.Model):
  def __init__(self) :
    super().__init__()
    self.Decoder1 = DecoderBlock(n_filters=512 , dropout=0.3 , block_name='Decoder1')
    self.Decoder2 = DecoderBlock(n_filters=256 , dropout=0.3 , block_name='Decoder2')
    self.Decoder3 = DecoderBlock(n_filters=128 , dropout=0.3 , block_name='Decoder3')
    self.Decoder4 = DecoderBlock(n_filters=64 , dropout=0.3 , block_name='Decoder4')
    self.final_conv = tf.keras.layers.Conv2D(19, kernel_size=(1,1), activation='softmax')
  def call(self , inputs , conv):
    f1, f2 , f3, f4 = conv
    c6 = self.Decoder1(inputs, f4 )
    c7 = self.Decoder2(c6,f3)
    c8 = self.Decoder3(c7,f2)
    c9= self.Decoder4(c8, f1)
    outputs = self.final_conv(c9)
    return outputs


In [ ]:
class  Unet (tf.keras.models.Model):
  def __init__(self):
    super().__init__()
    self.Encoder = Encoder()
    self.BottleNeck = BottleNeck()
    self.Decoder = Decoder()
  def call(self , inputs):
    encoder_output,convs = self.Encoder(inputs)
    bottleNeck_output = self.BottleNeck(encoder_output)
    output = self.Decoder(bottleNeck_output,convs)
    return output

***UNET ++***

In [ ]:
class Unet_plus_plus_convolutional_block(tf.keras.models.Model):
  def __init__(self, n_filters , block_name , activation = 'relu',kernel_size=(3,3)  ,stride = (2,2) ):
    super().__init__()
    self.con_layers = [tf.keras.layers.Conv2D(filters=n_filters , kernel_size= kernel_size , stride = stride , name=f"{block_name}_conv{i+1}",padding = 'same') for i in range (2)]
  def call(self, inputs) :
    x = inputs
    for layer in self.conv_layers:
      x=layer(x)
    return x

In [ ]:
class Encoder_Block_for_unt_plus_plus(tf.keras.models.Model):
  def __init__(self,  n_filters , pool_size , dropout,block_name):
    self.conv = Unet_plus_plus_convolutional_block(n_filters=n_filters, block_name=block_name)
    self.pool = tf.keras.layers.MaxPooling2D(pool_size=pool_size)
    self.dropout = tf.keras.layers.Dropout(rate =dropout)
  def call(self,inputs):
    f=self.conv(inputs)
    p = self.pool(f)
    p = self.dropout(p)
    return f ,  p

**Initilizating instaces of the models**

In [ ]:
with strategy.scope():
  Fcn =FCN()
  Unet = Unet()

NameError: name 'strategy' is not defined

**defining loss**

In [ ]:
import tensorflow as tf

with strategy.scope():
    class DiceLoss(tf.keras.losses.Loss):
        def __init__(self, smooth=1e-6):
            super().__init__()
            self.smooth = smooth

        def call(self, y_true, y_pred):

            y_true_f = tf.keras.backend.flatten(y_true)
            y_pred_f = tf.keras.backend.flatten(y_pred)

            intersection = tf.reduce_sum(y_true_f * y_pred_f)
            denominator = tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f)

            dice_loss = 1 - (2.0 * intersection + self.smooth) / (denominator + self.smooth)
            return dice_loss

with strategy.scope():
    class SemanticSegmentationLoss(tf.keras.losses.Loss):
        def __init__(self):
            super().__init__()
            self.dice_loss = DiceLoss()

        def call(self, y_true, y_pred):

            ce = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)
            dice_loss = self.dice_loss(y_true, y_pred)
            return ce + dice_loss


In [ ]:
with strategy.scope():
  class DeepSupervisionLoss(tf.keras.losses.Loss):
    def __init__(self, weights=None, smooth=1e-6):
      super().__init__()
      self.smooth = smooth
      self.weights = weights if weights else [1.0]
      self.ce_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    def dice_loss(self, y_true, y_pred):
      y_true_f = tf.keras.backend.flatten(tf.one_hot(tf.cast(y_true, tf.int32), depth=tf.shape(y_pred)[-1]))
      y_pred_f = tf.keras.backend.flatten(tf.nn.softmax(y_pred))
      intersection = tf.reduce_sum(y_true_f * y_pred_f)
      denominator = tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f)
      return 1 - (2.0 * intersection + self.smooth) / (denominator + self.smooth)

    def call(self, y_true, y_pred):
      total_loss = 0.0
      num_outputs = len(y_pred)
      for i in range(num_outputs):
        ce = self.ce_loss(y_true, y_pred[i])
        dice = self.dice_loss(y_true, y_pred[i])
        total_loss += self.weights[i] * (ce + dice)
      return total_loss


NameError: name 'strategy' is not defined

In [ ]:
with strategy.scope():
    optimizer = mixed_precision.LossScaleOptimizer(tf.keras.optimizers.Adam(learning_rate=1e-5), dynamic=True)